In [1]:
# header / imports
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sc3nb as scn
from sc3nb import Synth

In [2]:
sc = scn.startup()

<IPython.core.display.Javascript object>

Starting sclang process... 

Done.
Registering OSC /return callback in sclang... Done.
Loading default sc3nb SynthDefs... Done.
Booting SuperCollider Server... 

Done.


# Nodes (Synth and Group)

One of the most important objects in SuperCollider are Nodes.

To see all Nodes on the Server use

In [3]:
sc.server.dump_tree()

NODE TREE Group 0
   469762049 group
   402653185 group
   335544321 group
   268435457 group
   201326593 group
   134217729 group
   67108865 group
   1 group


You can also get a python object represenation via

In [4]:
root_node = sc.server.query_all_nodes()
root_node

Group(0)  {} children=[
  <Synth(-24) 's2'  {'freq': 1000.0, 'amp': 0.05000000074505806, 'num': 2.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>,
  Group(469762049) ~ {} children=[],
  Group(402653185) ~ {} children=[],
  Group(335544321) ~ {} children=[],
  Group(268435457) ~ {} children=[],
  Group(201326593) ~ {} children=[],
  Group(134217729) ~ {} children=[],
  Group(67108865) ~ {} children=[],
  Group(1) ~ {} children=[]]

In [5]:
root_node.children[-1]

Group(1) ~ {} children=[]

A node is either a [Synth](#Synth) or a [Group](#Group) of nodes

## Synth

### Create and control a Synth

Create a new Synth

In [6]:
synth = Synth(name="s2", controls={"freq": 100})
synth

<Synth(20001) 's2' s {'freq': 100}>

You can now hear the Synth playing and see it in the NodeTree

In [7]:
sc.server.default_group

Group(67108865) ~ {} children=[]

In [8]:
sc.server.query_all_nodes()

Group(0)  {} children=[
  <Synth(-24) 's2'  {'freq': 1000.0, 'amp': 0.05000000074505806, 'num': 2.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>,
  Group(469762049) ~ {} children=[],
  Group(402653185) ~ {} children=[],
  Group(335544321) ~ {} children=[],
  Group(268435457) ~ {} children=[],
  Group(201326593) ~ {} children=[],
  Group(134217729) ~ {} children=[],
  Group(67108865) ~ {} children=[
    <Synth(20001) 's2' ~ {'freq': 100.0, 'amp': 0.30000001192092896, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>],
  Group(1) ~ {} children=[]]

Free a synth

In [9]:
synth.free()

<Synth(20001) 's2' f {'freq': 100.0, 'amp': 0.30000001192092896, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>

Start the synth again

In [10]:
synth.new()

<Synth(20001) 's2' s {'freq': 100.0, 'amp': 0.30000001192092896, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>

In [11]:
synth.nodeid

20001

Calling `new` on an already running synth will cause a SuperCollider Server error

In [12]:
synth.new()

<Synth(20001) 's2' s {'freq': 100.0, 'amp': 0.30000001192092896, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>

Pause a synth

In [13]:
synth.run(False)

<Synth(20001) 's2' s {'freq': 100.0, 'amp': 0.30000001192092896, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>

In [14]:
synth

<Synth(20001) 's2' s {'freq': 100.0, 'amp': 0.30000001192092896, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>

Run a paused synth

In [15]:
synth.run() # default flag for run is True

<Synth(20001) 's2' s {'freq': 100.0, 'amp': 0.30000001192092896, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>

You can also wait for a Synth

In [16]:
synth_with_duration = scn.Synth("s1", dict(dur=2))
# wait for the Synth to finish playing
synth_with_duration.wait()

### Set / get Synth parameters

Set synth parameters, using any for the following `set` calls

    set(key, value, ...)
    set(list_of_keys_and_values])
    set(dict)

In [17]:
synth.set("freq", 200)

<Synth(20001) 's2' f {'freq': 200, 'amp': 0.30000001192092896, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>

In [18]:
synth.set(["freq", 30, "amp", 0.3])

<Synth(20001) 's2' f {'freq': 30, 'amp': 0.3, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>

In [19]:
synth.set({"freq": 130, "amp": 0.1})

<Synth(20001) 's2' f {'freq': 130, 'amp': 0.1, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>

The Synth does save its current control arguments in `current_controls`

In [20]:
synth.current_controls

{'freq': 130,
 'amp': 0.1,
 'num': 4.0,
 'pan': 0.0,
 'lg': 0.10000000149011612,
 'gate': 1.0}

However these are only cached values on the python object. 
Updating them will only affect the `new` call. See below why this can be useful.

In [21]:
synth.current_controls['freq'] = 440
synth

<Synth(20001) 's2' f {'freq': 440, 'amp': 0.1, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>

In [22]:
synth.free()
synth.new() # The new Synth will be created with freq = 440

<Synth(20001) 's2' s {'freq': 440, 'amp': 0.1, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>

To see what arguments can be set you can look at the `synth_desc`

In [23]:
synth.synth_desc

{'freq': SynthArgument(name='freq', rate='control', default=400.0),
 'amp': SynthArgument(name='amp', rate='control', default=0.30000001192092896),
 'num': SynthArgument(name='num', rate='control', default=4.0),
 'pan': SynthArgument(name='pan', rate='control', default=0.0),
 'lg': SynthArgument(name='lg', rate='control', default=0.10000000149011612),
 'gate': SynthArgument(name='gate', rate='control', default=1.0)}

You can use ``get`` to see the current value. This will request the current value from the SuperCollider audio server

In [24]:
synth.get("freq")

440.0

In [25]:
synth.get("pan")

0.0

This will also update the cached values in `current_args`

In [26]:
synth

<Synth(20001) 's2' ~ {'freq': 440.0, 'amp': 0.1, 'num': 4.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>

This is also possible directly with

In [27]:
synth.pan

0.0

Which can also be used for setting the argument

In [28]:
synth.pan = -1 

You can also query information about the Synth. Look at [Group](#Group) for more information about these values

In [29]:
synth.query()

SynthInfo(nodeid=20001, group=67108865, prev_nodeid=-1, next_nodeid=-1)

In [30]:
synth.free()

<Synth(20001) 's2' f {'freq': 440.0, 'amp': 0.1, 'num': 4.0, 'pan': -1, 'lg': 0.10000000149011612, 'gate': 1.0}>

Keep in mind that getting a synth value is always querying the server.

This means we need to receive a message which cant be done using the Bundler.
If you want to use relative values in a Bundler you should use the cached values from `current_args`

In [31]:
with sc.server.bundler() as bundle:
    synth.new({"freq": 600})
    for _ in range(100):
        synth.set(['freq', synth.current_controls['freq'] * 0.99])
        bundle.wait(0.05)
    synth.free()

In [32]:
synth.wait() # wait for synth

Some methods also allow getting the OSC Message instead of sending

In [33]:
synth.new(return_msg=True)

<OSCMessage("/s_new", ['s2', 20001, 0, 67108865, 'freq', 219.61940002441406, 'amp', 0.10000000149011612, 'num', 4.0, 'pan', -1, 'lg', 0.10000000149011612, 'gate', 1.0])>

Refer to the [OSC communication example notebook](../osc-communication-examples.ipynb) if you want to learn more about messages and bundles.

## Group

Nodes can be grouped and controlled together. This gives you mutliple advantages like

* controlling multiple Synth together
* specifying the execution order of the nodes. For more details look at [Order of nodes](#Order-of-Nodes)

The SuperCollider audio server scsynth does have one root Group with the Node ID 0.

In this root group each user got an default group were all the Nodes of the user (Synths and Groups) should be located

In [34]:
sc.server.query_all_nodes()

Group(0)  {} children=[
  <Synth(-24) 's2'  {'freq': 1000.0, 'amp': 0.05000000074505806, 'num': 2.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>,
  Group(469762049) ~ {} children=[],
  Group(402653185) ~ {} children=[],
  Group(335544321) ~ {} children=[],
  Group(268435457) ~ {} children=[],
  Group(201326593) ~ {} children=[],
  Group(134217729) ~ {} children=[],
  Group(67108865) ~ {} children=[],
  Group(1) ~ {} children=[]]

We have the following default Group

In [35]:
sc.server.default_group

Group(67108865) ~ {} children=[]

### Creating Groups

In [36]:
g0 = scn.Group()
g0

Group(20003) s {} children=[]

In [37]:
sc.server.dump_tree()

NODE TREE Group 0
   -24 s2
     freq: 1000 amp: 0.050000000745058 num: 2 pan: 0 lg: 0.10000000149012 gate: 1
   469762049 group
   402653185 group
   335544321 group
   268435457 group
   201326593 group
   134217729 group
   67108865 group
      20001 s2
        freq: 594 amp: 0.10000000149012 num: 4 pan: -1 lg: 0.10000000149012 gate: 1
      20003 group
   1 group


In [38]:
g0.free()

Group(20003) f {} children=[]

In [39]:
sc.server.dump_tree()

NODE TREE Group 0
   -24 s2
     freq: 1000 amp: 0.050000000745058 num: 2 pan: 0 lg: 0.10000000149012 gate: 1
   469762049 group
   402653185 group
   335544321 group
   268435457 group
   201326593 group
   134217729 group
   67108865 group
      20001 s2
        freq: 594 amp: 0.10000000149012 num: 4 pan: -1 lg: 0.10000000149012 gate: 1
   1 group


In [40]:
g0.new()

Group(20003) s {} children=[]

In [41]:
sc.server.query_all_nodes()

Group(0)  {} children=[
  <Synth(-24) 's2'  {'freq': 1000.0, 'amp': 0.05000000074505806, 'num': 2.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>,
  Group(469762049) ~ {} children=[],
  Group(402653185) ~ {} children=[],
  Group(335544321) ~ {} children=[],
  Group(268435457) ~ {} children=[],
  Group(201326593) ~ {} children=[],
  Group(134217729) ~ {} children=[],
  Group(67108865) ~ {} children=[
    Group(20003) ~ {} children=[],
    <Synth(20001) 's2' ~ {'freq': 588.0599975585938, 'amp': 0.10000000149011612, 'num': 4.0, 'pan': -1.0, 'lg': 0.10000000149011612, 'gate': 1.0}>],
  Group(1) ~ {} children=[]]

In [42]:
sc.server.default_group

Group(67108865) ~ {} children=[
  Group(20003) ~ {} children=[],
  <Synth(20001) 's2' ~ {'freq': 588.0599975585938, 'amp': 0.10000000149011612, 'num': 4.0, 'pan': -1.0, 'lg': 0.10000000149011612, 'gate': 1.0}>]

Create a Group in our new Group

In [43]:
g1 = scn.Group(target=g0)
g1

Group(20004) s {} children=[]

In [44]:
sc.server.query_all_nodes()

Group(0)  {} children=[
  <Synth(-24) 's2'  {'freq': 1000.0, 'amp': 0.05000000074505806, 'num': 2.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>,
  Group(469762049) ~ {} children=[],
  Group(402653185) ~ {} children=[],
  Group(335544321) ~ {} children=[],
  Group(268435457) ~ {} children=[],
  Group(201326593) ~ {} children=[],
  Group(134217729) ~ {} children=[],
  Group(67108865) ~ {} children=[
    Group(20003) ~ {} children=[ Group(20004) ~ {} children=[]],
    <Synth(20001) 's2' ~ {'freq': 582.1793823242188, 'amp': 0.10000000149011612, 'num': 4.0, 'pan': -1.0, 'lg': 0.10000000149011612, 'gate': 1.0}>],
  Group(1) ~ {} children=[]]

You can get information about you Group via

In [45]:
g0.query()

GroupInfo(nodeid=20003, group=67108865, prev_nodeid=-1, next_nodeid=20001, head=20004, tail=20004)

The query contains the same information as a Synth query and additionally the head and tail Node of this group.

Notice the special value `-1` meaning `None`.

In [46]:
g1.query()

GroupInfo(nodeid=20004, group=20003, prev_nodeid=-1, next_nodeid=-1, head=-1, tail=-1)

Free the node

In [47]:
g0.free()

Group(20003) f {} children=[ Group(20004) ~ {} children=[]]

In [48]:
sc.server.query_all_nodes()

Group(0)  {} children=[
  <Synth(-24) 's2'  {'freq': 1000.0, 'amp': 0.05000000074505806, 'num': 2.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>,
  Group(469762049) ~ {} children=[],
  Group(402653185) ~ {} children=[],
  Group(335544321) ~ {} children=[],
  Group(268435457) ~ {} children=[],
  Group(201326593) ~ {} children=[],
  Group(134217729) ~ {} children=[],
  Group(67108865) ~ {} children=[
    <Synth(20001) 's2' ~ {'freq': 576.3576049804688, 'amp': 0.10000000149011612, 'num': 4.0, 'pan': -1.0, 'lg': 0.10000000149011612, 'gate': 1.0}>],
  Group(1) ~ {} children=[]]

## Order of Nodes 



The execution of the Nodes does plays an important role. 
For more details look at the [SuperCollider Documentation](http://doc.sccode.org/Guides/Order-of-execution.html)

The Node placement of a Nodes can be controlled by the instantiation arguments

* `add_action` - An AddAction that specifies where to put Node relative to the target

In [49]:
list(scn.AddAction)

[<AddAction.TO_HEAD: 0>,
 <AddAction.TO_TAIL: 1>,
 <AddAction.BEFORE: 2>,
 <AddAction.AFTER: 3>,
 <AddAction.REPLACE: 4>]

* `target` - The target of the AddAction
* `group` - The group were the Node will be placed

### Example of Ordering Nodes

In [50]:
g0 = scn.Group()
g0

Group(20005) s {} children=[]

In [51]:
g1 = scn.Group(target=g0)
g1

Group(20006) s {} children=[]

In [52]:
s0 = scn.Synth(target=g0, controls={"freq": 200})

In [53]:
s1 = scn.Synth(add_action=scn.AddAction.BEFORE, target=s0, controls={"freq": 600})

In [54]:
s2 = scn.Synth(add_action=scn.AddAction.TO_TAIL, target=g1, controls={"freq": 1200})

In [55]:
sc.server.query_all_nodes()

Group(0)  {} children=[
  <Synth(-24) 's2'  {'freq': 1000.0, 'amp': 0.05000000074505806, 'num': 2.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>,
  Group(469762049) ~ {} children=[],
  Group(402653185) ~ {} children=[],
  Group(335544321) ~ {} children=[],
  Group(268435457) ~ {} children=[],
  Group(201326593) ~ {} children=[],
  Group(134217729) ~ {} children=[],
  Group(67108865) ~ {} children=[
    Group(20005) ~ {} children=[
      <Synth(20008) 'default' ~ {'freq': 600.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
      <Synth(20007) 'default' ~ {'freq': 200.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
      Group(20006) ~ {} children=[
        <Synth(20009) 'default' ~ {'freq': 1200.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>]],
    <Synth(20001) 's2' ~ {'freq': 570.5940551757812, 'amp': 0.10000000149011612, 'num': 4.0, 'pan': -1.0, 'lg': 0.10000000149011612, 'gate': 1.0}>],
  Group(1) ~ {} childr

In [56]:
s1.move(scn.AddAction.BEFORE, s2)

<Synth(20008) 'default' ~ {'freq': 600.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>

In [57]:
sc.server.query_all_nodes()

Group(0)  {} children=[
  <Synth(-24) 's2'  {'freq': 1000.0, 'amp': 0.05000000074505806, 'num': 2.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>,
  Group(469762049) ~ {} children=[],
  Group(402653185) ~ {} children=[],
  Group(335544321) ~ {} children=[],
  Group(268435457) ~ {} children=[],
  Group(201326593) ~ {} children=[],
  Group(134217729) ~ {} children=[],
  Group(67108865) ~ {} children=[
    Group(20005) ~ {} children=[
      <Synth(20007) 'default' ~ {'freq': 200.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
      Group(20006) ~ {} children=[
        <Synth(20008) 'default' ~ {'freq': 600.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
        <Synth(20009) 'default' ~ {'freq': 1200.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>]],
    <Synth(20001) 's2' ~ {'freq': 564.8880615234375, 'amp': 0.10000000149011612, 'num': 4.0, 'pan': -1.0, 'lg': 0.10000000149011612, 'gate': 1.0}>],
  Group(1) ~ {} chil

In [58]:
g0

Group(20005) ~ {} children=[
  <Synth(20007) 'default' ~ {'freq': 200.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
  Group(20006) ~ {} children=[
    <Synth(20008) 'default' ~ {'freq': 600.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
    <Synth(20009) 'default' ~ {'freq': 1200.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>]]

In [59]:
g0.run(False)

Group(20005) ~ {} children=[
  <Synth(20007) 'default' ~ {'freq': 200.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
  Group(20006) ~ {} children=[
    <Synth(20008) 'default' ~ {'freq': 600.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
    <Synth(20009) 'default' ~ {'freq': 1200.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>]]

In [60]:
g0.run(True)

Group(20005) ~ {} children=[
  <Synth(20007) 'default' ~ {'freq': 200.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
  Group(20006) ~ {} children=[
    <Synth(20008) 'default' ~ {'freq': 600.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
    <Synth(20009) 'default' ~ {'freq': 1200.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>]]

In [61]:
g1.run(False)
g1.query_tree()

Group(20006) ~ {} children=[
  <Synth(20008) 'default' ~ {'freq': 600.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
  <Synth(20009) 'default' ~ {'freq': 1200.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>]

In [62]:
s0.freq, s1.freq, s2.freq

(200.0, 600.0, 1200.0)

In [63]:
g0.set("freq", 100)
s0.freq, s1.freq, s2.freq

(100.0, 100.0, 100.0)

In [64]:
sc.server.dump_tree()

NODE TREE Group 0
   -24 s2
     freq: 1000 amp: 0.050000000745058 num: 2 pan: 0 lg: 0.10000000149012 gate: 1
   469762049 group
   402653185 group
   335544321 group
   268435457 group
   201326593 group
   134217729 group
   67108865 group
      20005 group
         20007 default
           out: 0 freq: 100 amp: 0.10000000149012 pan: 0 gate: 1
         20006 group
            20008 default
              out: 0 freq: 100 amp: 0.10000000149012 pan: 0 gate: 1
            20009 default
              out: 0 freq: 100 amp: 0.10000000149012 pan: 0 gate: 1
      20001 s2
        freq: 548.1103515625 amp: 0.10000000149012 num: 4 pan: -1 lg: 0.10000000149012 gate: 1
   1 group


In [65]:
g1.run(True)

Group(20006) ~ {} children=[
  <Synth(20008) 'default' ~ {'freq': 100.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
  <Synth(20009) 'default' ~ {'freq': 100.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>]

In [66]:
sc.server.query_all_nodes()

Group(0)  {} children=[
  <Synth(-24) 's2'  {'freq': 1000.0, 'amp': 0.05000000074505806, 'num': 2.0, 'pan': 0.0, 'lg': 0.10000000149011612, 'gate': 1.0}>,
  Group(469762049) ~ {} children=[],
  Group(402653185) ~ {} children=[],
  Group(335544321) ~ {} children=[],
  Group(268435457) ~ {} children=[],
  Group(201326593) ~ {} children=[],
  Group(134217729) ~ {} children=[],
  Group(67108865) ~ {} children=[
    Group(20005) ~ {'freq': 100} children=[
      <Synth(20007) 'default' ~ {'freq': 100.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
      Group(20006) ~ {} children=[
        <Synth(20008) 'default' ~ {'freq': 100.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
        <Synth(20009) 'default' ~ {'freq': 100.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>]],
    <Synth(20001) 's2' ~ {'freq': 542.6292724609375, 'amp': 0.10000000149011612, 'num': 4.0, 'pan': -1.0, 'lg': 0.10000000149011612, 'gate': 1.0}>],
  Group(1)

In [67]:
g1.set("freq", 440)

s0.freq, s1.freq, s2.freq

(100.0, 440.0, 440.0)

In [68]:
g0.query_tree()

Group(20005) ~ {'freq': 100} children=[
  <Synth(20007) 'default' ~ {'freq': 100.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
  Group(20006) ~ {'freq': 440} children=[
    <Synth(20008) 'default' ~ {'freq': 440.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
    <Synth(20009) 'default' ~ {'freq': 440.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>]]

In [69]:
g0.move_node_to_tail(s1)

Group(20005) ~ {'freq': 100} children=[
  <Synth(20007) 'default' ~ {'freq': 100.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
  Group(20006) ~ {'freq': 440} children=[
    <Synth(20008) 'default' ~ {'freq': 440.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
    <Synth(20009) 'default' ~ {'freq': 440.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>]]

In [70]:
g0.dump_tree()

Group(20005) ~ {'freq': 100} children=[
  <Synth(20007) 'default' ~ {'freq': 100.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
  Group(20006) ~ {'freq': 440} children=[
    <Synth(20008) 'default' ~ {'freq': 440.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
    <Synth(20009) 'default' ~ {'freq': 440.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>]]

In [71]:
g1.set("freq", 800)

s0.freq, s1.freq, s2.freq

(100.0, 440.0, 800.0)

In [72]:
sc.server.default_group.query_tree()

Group(67108865) ~ {} children=[
  Group(20005) ~ {'freq': 100} children=[
    <Synth(20007) 'default' ~ {'freq': 100.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>,
    Group(20006) ~ {'freq': 800} children=[
      <Synth(20009) 'default' ~ {'freq': 800.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>],
    <Synth(20008) 'default' ~ {'freq': 440.0, 'out': 0.0, 'amp': 0.10000000149011612, 'pan': 0.0, 'gate': 1.0}>],
  <Synth(20001) 's2' ~ {'freq': 542.6292724609375, 'amp': 0.10000000149011612, 'num': 4.0, 'pan': -1.0, 'lg': 0.10000000149011612, 'gate': 1.0}>]

In [73]:
sc.server.free_all()
sc.server.dump_tree()

NODE TREE Group 0
   469762049 group
   402653185 group
   335544321 group
   268435457 group
   201326593 group
   134217729 group
   67108865 group
   1 group


In [74]:
sc.exit()